In [39]:
## IMPORTS ##
import scipy.sparse
import numpy as np
import matplotlib.pyplot as pyplot
import sklearn.metrics as mtc
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import MaxAbsScaler
from sklearn.neural_network import BernoulliRBM
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.decomposition import PCA
from sklearn.decomposition import TruncatedSVD
from sklearn.model_selection import train_test_split

In [81]:
X_tr = scipy.sparse.load_npz('../../Datasets/dataset/X_tr.npz')
X_tst = scipy.sparse.load_npz('../../Datasets/dataset/X_tst.npz')
y_tr = np.load('../../Datasets/dataset/y_tr.npy')
y_tst = np.load('../../Datasets/dataset/y_tst.npy')
# Se binarizan las caracteristicas 
y_tr=MultiLabelBinarizer().fit_transform(y_tr)
y_tst=MultiLabelBinarizer().fit_transform(y_tst)
# Escalado 
scaler = MaxAbsScaler().fit(X_tr)
X_tr=scaler.transform(X_tr)
X_tst=scaler.transform(X_tst)


In [73]:
# REDUCCIÓN DE DIMENSIÓN PRIMERA TÉCNICA
svd = TruncatedSVD(n_components=17, n_iter=10, random_state=42)
X_tr = svd.fit_transform(X_tr)  
X_tst = svd.transform(X_tst)  
print(svd.explained_variance_ratio_)
print(svd.explained_variance_ratio_.sum())
print(svd.singular_values_)



ValueError: Expected 2D array, got 1D array instead:
array=[].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.

In [82]:
#REDUCCION TAMAÑO TRAINING Y ELIMINAR CARACTERISTICAS INSERVIBLES
X_aux, __, y_aux, __ = train_test_split(X_tr, y_tr, test_size=0.80, random_state=42)
X_tr=np.array([])
print(np.shape(X_aux))
for i in range(np.shape(X_aux)[1]):
    if X_aux[:,i].count_nonzero()> 100:
        np.append(X_tr,X_aux[:,i])
        print(i)
print(np.shape(X_tr)) 

(18789, 44507)
183
216
405
407
411
451
456
479
480
496
509
523
524
530
550
554
557
562
589
598
599
605
607
623
626
638
639
642
645
692
718
728
734
740
747
751
752
757
814
816
817
832
838
839
841
844
845
858
864
868
874
875
876
882
890
904
906
925
926
931
939
971
979
980
995
1015
1016
1028
1039
1043
1044
1052
1053
1058
1087
1094
1110
1146
1165
1198
1223
1226
1240
1246
1279
1282
1284
1349
1356
1357
1359
1365
1368
1373
1387
1397
1401
1407
1422
1429
1430
1431
1465
1480
1485
1506
1560
1590
1594
1602
1633
1635
1649
1658
1673
1686
1715
1730
1758
1759
1760
1765
1783
1802
1889
1893
1894
1897
1901
1902
1904
1920
1924
1930
1944
1962
2003
2006
2011
2012
2021
2036
2038
2040
2072
2074
2145
2147
2158
2201
2204
2206
2209
2237
2249
2269
2274
2278
2388
2405
2406
2408
2412
2450
2451
2490
2500
2508
2522
2533
2556
2567
2589
2618
2620
2627
2640
2661
2707
2740
2752
2794
2797
2803
2806
2808
2827
2833
2865
2868
2904
2908
2909
2917
2941
2948
2954
2965
2967
2972
2976
2977
2991
2992
3029
3069
3101
3119
3142
3153


22173
22176
22183
22184
22188
22199
22201
22202
22205
22215
22216
22217
22224
22231
22252
22254
22255
22268
22270
22277
22280
22291
22294
22298
22305
22319
22335
22337
22347
22359
22361
22371
22384
22385
22389
22393
22426
22449
22466
22468
22471
22476
22480
22507
22509
22518
22522
22554
22556
22558
22566
22572
22573
22574
22584
22590
22591
22685
22688
22690
22736
22738
22740
22746
22751
22753
22754
22765
22767
22792
22800
22806
22819
22820
22836
22923
23002
23009
23041
23084
23120
23176
23358
23361
23373
23392
23407
23412
23415
23425
23437
23440
23449
23458
23460
23461
23462
23469
23799
23800
23803
23860
23887
23908
23998
24007
24049
24106
24117
24129
24146
24149
24164
24165
24195
24208
24210
24215
24223
24246
24281
24304
24314
24346
24348
24353
24357
24363
24383
24385
24387
24408
24414
24419
24452
24453
24470
24475
24478
24492
24496
24512
24529
24555
24559
24597
24632
24641
24645
24668
24702
24715
24722
24781
24784
24785
24814
24817
24826
24851
24852
24969
24981
24982
24983
24996
2503

In [95]:
X_tr=scipy.sparse.csr_matrix([])
print(X_aux[:,44453])
print(scipy.sparse.hstack(X_tr,X_aux))

  (10, 0)	0.08030441216713528
  (11, 0)	0.16420218091071206
  (16, 0)	0.3132613512850638
  (24, 0)	0.6929951260727445
  (25, 0)	0.5483418316372972
  (31, 0)	0.06914285275255104
  (53, 0)	0.5018961400705976
  (100, 0)	0.06258391356850823
  (103, 0)	0.04438403619294346
  (111, 0)	0.11074891564456348
  (113, 0)	0.5421584855304279
  (121, 0)	0.218665719975425
  (147, 0)	0.07967331154512505
  (155, 0)	0.06353769341837896
  (172, 0)	0.05937696787591301
  (181, 0)	0.09881600246457098
  (196, 0)	0.15751647125683202
  (224, 0)	0.03982202939648888
  (234, 0)	0.05504808530482184
  (235, 0)	0.35475402044521487
  (236, 0)	0.07009768170105467
  (242, 0)	0.11830494016654981
  (249, 0)	0.044974535261176965
  (269, 0)	0.06484360506241311
  (273, 0)	0.0667907071018738
  :	:
  (18526, 0)	0.12977726687412994
  (18527, 0)	0.07723854644854565
  (18530, 0)	0.08933283112387817
  (18533, 0)	0.11217068858928744
  (18539, 0)	0.12518262834101143
  (18559, 0)	0.09025095926688192
  (18568, 0)	0.23974796192472306
  

ValueError: blocks must be 2-D

In [57]:
## Clasificador de regresión logística
classif = OneVsRestClassifier(LogisticRegression())
parameters = {
    "estimator__C": [0.0001,0.001,0.01,0.1,1,10,100,1000],
    "estimator__solver": ["liblinear"]
}

model_tunning = GridSearchCV(classif, param_grid=parameters,cv=4)

model_tunning.fit(X_tr, y_tr)

print (model_tunning.best_score_)
print (model_tunning.best_params_)


KeyboardInterrupt: 

In [37]:
y_pred=model_tunning.predict(X_tst)


In [38]:
accuracy=mtc.accuracy_score(y_tst,y_pred)
hamming=mtc.hamming_loss(y_tst,y_pred)
print(accuracy)
print(hamming)


0.15160022459292533
0.04341624049653247


In [ ]:
#RBM -- SE IMPLEMENTARÁ CON UN PIPELINE PERO AÚN FALTA ENTENDER MEJOR EL FUNCIONAMIENTO DE BERNOULLIRBM -- 
#-- NO EJECUTAR-- #
logistic = OneVsRestClassifier(LogisticRegression())
rbm = BernoulliRBM(random_state=0, verbose=True)
rbm_features_classifier = Pipeline(steps=[('rbm', rbm), ('logistic', logistic)])
rbm.learning_rate = 0.06
rbm.n_iter = 20
rbm.n_components = 100
rbm_features_classifier.fit(X_tr, y_tr)
Y_pred2 = rbm_features_classifier.predict(X_tst)
